In [12]:
import pandas as pd
import chardet
import glob
import os
all_files = glob.glob(os.path.join("loc221csv", "2022-1 UNLOCODE CodeListPart*.csv"))
all_files

['loc221csv/2022-1 UNLOCODE CodeListPart2.csv',
 'loc221csv/2022-1 UNLOCODE CodeListPart3.csv',
 'loc221csv/2022-1 UNLOCODE CodeListPart1.csv']

In [13]:
dfs = []
for filename in all_files:
    with open(filename, 'rb') as f:
        result = chardet.detect(f.read())
        print(filename, result["encoding"])
        dfs.append(pd.read_csv(filename, encoding=result["encoding"], header=None, keep_default_na=False))

loc221csv/2022-1 UNLOCODE CodeListPart2.csv Windows-1252
loc221csv/2022-1 UNLOCODE CodeListPart3.csv ISO-8859-1
loc221csv/2022-1 UNLOCODE CodeListPart1.csv ISO-8859-1


In [14]:
df = pd.concat(dfs)
df.columns = ["Ch", "Country Code", "Location Code", "Name", "NameWoDiacritics", "SubDiv", "Function", "Status", "Date", "IATA", "Coordinates", "Remarks"]
df = df[df["Location Code"] != ""] # Remove row if location code is empty
df

,Ch,Country Code,Location Code,Name,NameWoDiacritics,SubDiv,Function,Status,Date,IATA,Coordinates,Remarks
1,,GA,AKE,Akieni,Akieni,,---4----,AI,9601,,,
2,,GA,AWE,Alowe,Alowe,,---4----,RQ,0901,,0045S 00927E,
3,,GA,BEL,Belleville,Belleville,,--3-----,RL,0607,,0002N 01102E,
4,,GA,BWN,Biawonque,Biawonque,,---4----,RQ,0901,,0040S 00927E,
5,,GA,BMM,Bitam,Bitam,,---4----,AI,9601,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
54656,,FR,ZTG,Zetting,Zetting,57,--3-----,RL,1901,,4905N 00708E,
54657,,FR,ZL2,Zilia,Zilia,2B,-----6--,RL,1901,,4231N 00854E,
54658,,FR,ZWL,Zinswiller,Zinswiller,67,--3-----,RL,1901,,4855N 00735E,
54659,,FR,PCZ,Zoteux,Zoteux,62,-----6--,RL,1001,,5037N 00153E,


In [15]:
# Preprocessing
df["code"] = df["Country Code"] + df["Location Code"]
df = df.rename(columns={
    "Name": "name",
    "Function": "function"
})
df = df[["name", "code", "function"]]
df

,name,code,function
1,Akieni,GAAKE,---4----
2,Alowe,GAAWE,---4----
3,Belleville,GABEL,--3-----
4,Biawonque,GABWN,---4----
5,Bitam,GABMM,---4----
...,...,...,...
54656,Zetting,FRZTG,--3-----
54657,Zilia,FRZL2,-----6--
54658,Zinswiller,FRZWL,--3-----
54659,Zoteux,FRPCZ,-----6--


In [16]:
source_df_sea = df[df["function"].str.contains("1")]
source_df_rail = df[df["function"].str.contains("2")]
source_df_road = df[df["function"].str.contains("3")]
source_df_air = df[df["function"].str.contains("4")]
source_df_dry = df[df["function"].str.contains("6")]

In [17]:
df_air = pd.read_json("old/airports.json")
df_air

,name,code
0,Andorra la Vella,ADALV
1,Abu Dhabi,AEAUH
2,Al Ain,AEAAN
3,Al Dhafra,AEDHF
4,Al Fujayrah,AEFJR
...,...,...
8869,Kariba,ZWKAB
8870,Mahenye,ZWMJW
8871,Masvingo,ZWMVZ
8872,Mutare,ZWUTA


In [18]:
df_sea = pd.read_json("old/seaports.json")
df_sea

,name,code
0,Abu al Bukhoosh,AEABU
1,Abu Dhabi,AEAUH
2,Abu Musa,AEAMU
3,Ahmed Bin Rashid Port,AEARP
4,Ajman,AEAJM
...,...,...
18504,Chegutu,ZWCHE
18505,Filabusi,ZWFLU
18506,Gwanda,ZWGWA
18507,Rusape,ZWRSP


In [19]:
import re
from tqdm.notebook import tqdm

def process(old_df, new_df, function):
    print("Before", function, len(old_df))

    with tqdm(total=new_df.shape[0]) as bar:
        for idx, row in new_df.iterrows():
            bar.update(1)
            if len(old_df[old_df["code"].eq(row["code"])]) == 0:
                old_df.loc[len(old_df)] = [row["name"], row["code"]]

    old_df = old_df.sort_values(by=['code'])
    print("After", function, len(old_df))
    print("--------------")
    return old_df

new_df_sea = process(df_sea.copy(), source_df_sea, "Sea")
new_df_rail = process(pd.DataFrame(columns=["name", "code"]), source_df_rail, "Rail")
new_df_road = process(pd.DataFrame(columns=["name", "code"]), source_df_road, "Road")
new_df_air = process(df_air.copy(), source_df_air, "Air")
new_df_dry = process(pd.DataFrame(columns=["name", "code"]), source_df_dry, "Dry")

Before Sea 18509


  0%|          | 0/17510 [00:00<?, ?it/s]

After Sea 18799
--------------
Before Rail 0


  0%|          | 0/13103 [00:00<?, ?it/s]

After Rail 13081
--------------
Before Road 0


...

After Road 89738
--------------
Before Air 8874


  0%|          | 0/9053 [00:00<?, ?it/s]

After Air 9201
--------------
Before Dry 0


  0%|          | 0/15129 [00:00<?, ?it/s]

After Dry 15127
--------------


In [20]:
# Validate some data
assert len(new_df_air[new_df_air["code"].eq("ADALV")]) > 0
assert len(new_df_air[new_df_air["code"].eq("BRACU")]) > 0
assert len(new_df_road[new_df_road["code"].eq("ADCAN")]) > 0
assert len(new_df_road[new_df_road["code"].eq("ADENC")]) > 0

In [21]:
new_df_sea.to_json("new/seaports.json", orient='records')
new_df_rail.to_json("new/rails.json", orient='records')
new_df_road.to_json("new/roads.json", orient='records')
new_df_air.to_json("new/airports.json", orient='records')
new_df_dry.to_json("new/dryports.json", orient='records')